In [ ]:
#Installing the XGBoost package
!pip install xgboost

In [ ]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import xgboost as xg
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from openpyxl import load_workbook

In [ ]:
#Reading the excel-document into a df
df = pd.read_excel('dataset_name.xlsx', Sheet_name = 'Sheet_name') 

#Choosing which columns to keep and saving them in a new df
columns_to_keep = ['Column 1', 'Column 2', 'Column 3', 'Column 4', 'Column 5', '...']
df2 = df[columns_to_keep].copy()

In [ ]:
#Splitting df into two based on hole ID
df2.loc[:, 'Hole_id_stat_str'] = df2['Hole_id_stat'].astype(str) #Making sure the IDs are strings

#Defining the unseen column(s)
columns_unseen = ['Column']
#Splitting into train and unseen data based on column
df_train = df2[~df2['Hole_id_stat_str'].isin(columns_unseen)] 
df_unseen = df2[df2['Hole_id_stat_str'].isin(columns_unseen)] 


In [ ]:
#Defining which columns to keep av model input
X = df_train.drop(columns = ['Column 1', 'Column 2', 'Column 3'])

#Defining which columns are the model outputs
y = df_train['Column 4']

#Defining inputs and outputs for unseen dataset
X_unseen = df_unseen.drop(columns = ['Column 1', 'Column 2', 'Column 3')
y_unseen = df_unseen['Column 4'] 

In [ ]:
#Using K-Fold cross-validation to split the data into 5 folds
k_fold = KFold(n_splits=5, random_state=66, shuffle=True)

#Defining a pipeline that standardizes the data before fitting an XGBoost regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),  
    ('xg', xg.XGBRegressor(        
        n_estimators=40, 
        max_depth=7, 
        random_state=40, 
        objective='reg:squarederror' 
    ))
])

#Creating empty lists to store evaluation metrics for each fold
mae_scores_train, mse_scores_train, r2_scores_train = [], [], []
mae_scores_test, mse_scores_test, r2_scores_test = [], [], []

#Looping over each train/test split from K-Fold
for train_index, test_index in k_fold.split(X, y):
    #Splitting the data based on the current fold
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #Skipping the fold if the test target only has one unique value
    if len(np.unique(y_test)) == 1:
        print("Skipping this fold due to only one class in test set.")
        continue

    #Fitting the pipeline on the training data
    pipeline.fit(X_train, y_train)

    #Predicting on both training and test sets
    pred_train = pipeline.predict(X_train)
    pred_test = pipeline.predict(X_test)

    #Calculating metrics on the training data
    mae_train = mean_absolute_error(y_train, pred_train)
    mse_train = mean_squared_error(y_train, pred_train)
    r2_train = r2_score(y_train, pred_train)

    #Calculating metrics on the test data
    mae_test = mean_absolute_error(y_test, pred_test)
    mse_test = mean_squared_error(y_test, pred_test)
    r2_test = r2_score(y_test, pred_test)

    #Saving the metrics for this fold
    mae_scores_train.append(mae_train)
    mse_scores_train.append(mse_train)
    r2_scores_train.append(r2_train)

    mae_scores_test.append(mae_test)
    mse_scores_test.append(mse_test)
    r2_scores_test.append(r2_test)

    #Printing metrics for the fold
    print(f"Train -> MAE: {mae_train:.6f}, MSE: {mse_train:.6f}, R²: {r2_train:.6f}")
    print(f"Test  -> MAE: {mae_test:.6f}, MSE: {mse_test:.6f}, R²: {r2_test:.6f}\n")

#Printing the average metrics across all folds
print("\nAverage results from all folds:")
print(f"Train Mean MAE: {np.mean(mae_scores_train):.6f}, Test Mean MAE: {np.mean(mae_scores_test):.6f}")
print(f"Train Mean MSE: {np.mean(mse_scores_train):.6f}, Test Mean MSE: {np.mean(mse_scores_test):.6f}")
print(f"Train Mean R²: {np.mean(r2_scores_train):.6f}, Test Mean R²: {np.mean(r2_scores_test):.6f}")

In [ ]:
#Defining the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),         
    ('xg', xg.XGBRegressor(random_state=66))  
])

#Defining the hyperparameter grid to tune the XGBoost model
parameters_grid = {
    'xg__n_estimators': [500, 700],           
    'xg__max_depth': [3, 4],                 
    'xg__learning_rate': [0.05],              
    'xg__subsample': [0.7],                   
    'xg__colsample_bytree': [0.8],            
    'xg__gamma': [1, 2],                      
    'xg__reg_alpha': [1, 3, 5],               
    'xg__reg_lambda': [6, 10],                
    'xg__min_child_weight': [1, 5, 10]        
}

#Running grid search cross-validation to find the best hyperparameters
CV_XGB = GridSearchCV(
    estimator=pipeline,                      
    param_grid=parameters_grid,              
    cv=k_fold,                               
    scoring='neg_mean_squared_error',        
    n_jobs=-1                               
)

#Fitting the GridSearchCV to the full dataset
CV_XGB.fit(X, y)

#Printing out the best parameter combination found
print('Best parameters: ', CV_XGB.best_params_)


In [ ]:
#Saving the best hyperparameters
best_params = {key.replace("xg__", ""): value for key, value in CV_XGB.best_params_.items()}

#Retrieve the best pipeline from GridSearchCV
best_pipeline = CV_XGB.best_estimator_

#Making predictions on the training and test sets 
y_train_pred = best_pipeline.predict(X_train)
y_test_pred = best_pipeline.predict(X_test)

#Evaluating the model on the training set
mse_train = mean_squared_error(y_train, y_train_pred)  
r2_train = r2_score(y_train, y_train_pred)             

#Evaluating the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

#Printing the evaluation metrics
print(f"Train Mean Squared Error: {mse_train:.4f}")
print(f"Train R-squared: {r2_train:.4f}")
print(f"Test Mean Squared Error: {mse_test:.4f}")
print(f"Test R-squared: {r2_test:.4f}")


In [ ]:
#Appling the fitted scaler from the pipeline to the unseen data
X_unseen_scaled = best_pipeline.named_steps['scaler'].transform(X_unseen)

#Using the fitted XGBoost model from the pipeline to predict on unseen data
y_unseen_pred = best_pipeline.named_steps['xg'].predict(X_unseen_scaled)

#Evaluating the model performance on the unseen data
mse_unseen = mean_squared_error(y_unseen, y_unseen_pred)  
r2_unseen = r2_score(y_unseen, y_unseen_pred)             

#Printing the performance metrics for the unseen dataset
print(f'Unseen Mean Squared Error: {mse_unseen:.4f}')
print(f'Unseen R-squared: {r2_unseen:.4f}')


In [ ]:
#Defining a function to log model evaluation metrics and hyperparameters to an Excel-file of chosen name
def log_results(model_name, params, mse_train, r2_train, mse_test, r2_test, mse_unseen, r2_unseen, filename="Results.xlsx"):
    """ Logs model results in separate sheets within the same Excel file, ensuring appending works correctly. """

    #Creating DataFrame for this model run
    result = pd.DataFrame([{
        **params,  
        "MSE_Train": mse_train,
        "R2_Train": r2_train,
        "MSE_Test": mse_test,
        "R2_Test": r2_test,
        "MSE_Unseen": mse_unseen,
        "R2_Unseen": r2_unseen
    }])

    #Introducing a short delay to avoid file conflicts if running in multiple notebooks
    time.sleep(1)

    #Checking if the file exists
    file_exists = os.path.exists(filename)

    if not file_exists:
        #If file doesn't exist, creating a new one
        with pd.ExcelWriter(filename, engine="openpyxl", mode="w") as writer:
            result.to_excel(writer, sheet_name=model_name, index=False)
    else:
        #If file exists, loading it properly before appending
        try:
            with pd.ExcelWriter(filename, engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
                #Reading existing sheet
                try:
                    existing_df = pd.read_excel(filename, sheet_name=model_name, engine="openpyxl")
                    df_combined = pd.concat([existing_df, result], ignore_index=True)
                except (FileNotFoundError, ValueError):
                    df_combined = result  #If sheet does not exist, creating it

                #Saving results
                df_combined.to_excel(writer, sheet_name=model_name, index=False)

        except PermissionError:
            #Printing error to warn user
            print(f"Error: Close the Excel file ({filename}) before running the script again.")

    #Printing a confirmation to ensure user results are logged
    print(f"Logged results for {model_name}: Train MSE={mse_train:.4f}, Test MSE={mse_test:.4f}, Unseen MSE={mse_unseen:.4f}")


In [ ]:
#Using the function to log results from RF classifier model
log_results(
    model_name="XGBoost_stat",
    params=best_params,
    mse_train=mse_train,  # Example values
    r2_train=r2_train,
    mse_test=mse_test,
    r2_test=r2_test,
    mse_unseen=mse_unseen,
    r2_unseen=r2_unseen
)